In [1]:
from keras.models import load_model
from pkg_resources import resource_filename
from utils2 import one_hot_encode
import numpy as np

import datetime as dt
import h5py
import numpy as np
import os
import fnmatch
import time
import re
from sklearn.metrics import average_precision_score
from pandas import DataFrame
import pandas as pd
import random

In [2]:
paths = ('snp_data/spliceai{}.h5'.format(x) for x in [1,2,3,4,5])
spliceai_models=[load_model(resource_filename('spliceai', x),compile=False) for x in paths]

In [3]:
def make_random_seq(seq_len):
    
    random_seq=''
    for _ in range(seq_len):
        xx=random.random()
        
        if xx<0.25:
            random_seq+='A'
        elif xx<0.5:
            random_seq+='T'
        elif xx<0.75:
            random_seq+='G'
        else:
            random_seq+='C'
    
    return random_seq


In [4]:
left_context='tgcaggccgcagtgagctatgatcagcttgggcgactgagcgagaccctgtctctaaaacaaacacacaaGTccgggcgcggtggctcatgcctgtaatcttagcactttgggaggccgaggtgggcggatcacgaggtcaagaaatcgagaccatcctggccaacatggtgaaaccccgtctctactaaaaatacaaaaattagctgggcgtggtggtgcgcgcctgtagtcccagctactcgggaggctgaggcaggagaatcgcttgaacccgggaggcagaggttgcagtgagccgagatcgtgccactgcactccagcctggcgacagagtgagactccgtctcagaacaaacaaacaaaaggatagaaaGGCGAGCACAAATATTCCCAATTCATAACACTCCCTCGCACTGTCAATGCCCCAGACACGCGCTATCATCTCTAGCAAACTCCCCCAGGCGCCTGCAGGATGGGTTAAGGAAGGCGACGAGCACCAGCTGCCCTGCTGAGGCTGTCCCGACGTCACATGATTCTCCAATCACATGATCCCTAGAAATGGGGTGTGGGGCGAGAGGAAGCAGGGAGGAGAGTGATTTGAGTAGAAAAGAAACACAGCATTCCAGGCTGGCCCCACCTCTATATTGATAAGTAGCCAATGGGAGCGGGTAGCCCTGATCCCTGGCCAATGGAAACTGAGGTAGGCGGGTCATCGCGCTGGGGTCTGTAGTCTGAGCGCTACCCGGTTGCTGCTGCCCAAGGACCGCGGAGTCGGACGCAGGTAGGAGAGCGGCCGCGCAGACCTCTCGCCTGCTCCTGCCCAGGGGCCCGCCAGGGCCATGTGAGCTTGAGGTTCCCCTGGAGTCTCAGCCGGAGACAACAGAAGAACCGCTTACTGAAACTCCTTGGGGGTTCTGATACACTAGGGGGAGTTTTATGGGAAAGAGGAAGCAGTAATTGCAGTGACGCCCCGTTAGAAGGGGCTTTCTACCTCCCCAGCATTCCCCCAAAGCAGGGACCACACCATTCTTGACCCAGCTCCACCCCTGTCGGTAGGTGCTGGCTTCTTCCCCTCTCCTGGTGGTGGTGGGTGGTTCCCGCGGCGGCCTGGAGCCGGAGGGGCGCGCGACCCTGGGCTGGGAGCTCCGAGGGCCTGGGAACGAGACCTGAGACCTTGGCTTCTCGAAGGTAGTAGGGACTTGGGAGTGGTGACTGAACCTGGTCTGGCTCCTCCTTACTTCCTCTTGTTGCGGGTGGGACGAGCTAGCTTCCGCCTCTCCCAGCCACTTTTTCCTGCTCATTTGCAGCTAGGTTGGCTCCCCTTTTGGGAATTTCCTCTCCCCTTGGCACTCGGAGTTGGGGGGTGCCACCTAGTGGAAGATAACGGAGCTAGGGTCTTGAAGAGGCTGCTGTCCCCTCTGGCTGTTTTGGCGGTGTAGGGTGGCATGAGAGACTGCGACTCGCCTCCTCATCCCTGTTTCTGTATGCGAGTGCTTGTATTCAGTAGAAGCATACACTATACTCCCTCAATTTAGGGTAAACAGGAGGGGCCACATGCACAGGTAATTCACCAGGGAGCCGAACACTCCTGTGCAGACAGACTCCCCTTCCCAGCAAGCCATGGCAGCGGACAGCCTGCTGAGAACACCCAGGAAGCAGGCGGTGCCAGCTGCAGGTGCTTTGCCTGGGAGCTGTGGGGCTGAGGAGAGGGTCCACTGTCCAGGACCAGTGAACTTCATCCTTATCTGTCCAGGAGGTGGCCTCTTGGGGATGCTGAGTTAGGGGAGGGGCACTTGAGGAAAGCCAGGTGGAGCAGAGAGGATGTGAGTGACTGGGTGGGTGAGATTTCCTGCCCCTCCCCCCGCAGTGGtatccacacctagactcgtggggtaactgaggcacagacagagagcaacttctcaggccctcacagttggcaattctaggattaggacccaagtgcgattttcaggcagtccctgtaccctgtttctgttgtacctgttgcaccATTCCCAGGCACTGCCCATCGTGCCACTAGTGATATGAACCCAGGTCCAATACGCTCTGGGGCCATCAAAGCCTGACGTCACCATGACCTGATGTGTGACGTGTTATaggtgtcccttggtatcttcacggaactggttccaggaccccaaaatctgtgggtgctcaagcccctgagataaaatggtgtaatatttgcatataacctatacatactttaaatcatttctagattacttatacctaatacaatggaaatgacatgtcggctgggcgtggtggctcatgcctgtaatcccaccactttgggaggccgtggcaggtggatcacctgaggtctggagtttgagaccagcctgaccaacatggtgaaacccccatctctactaaaaatacaaaaattagccaggtgtggtagcgcacacctataatcccacctacttgggaggctgaggcaggagaattgcttgaacctgggaggcggagttcgcagtaagctgagatcgcgccactgtactacagcctgggtgacagagcaggactccatctcaaaaaaaaaagagaaaaagaaaaagaaatgccatgtaaatagttgtgatcctgaattgtttagggaataataagaaagaactatctgtagatgttcagtatagatgcacccatcGTAAGCCTAACTACATTGTATAACTCAGCAACGATGTAACATTTTCAGGGGtttttttgttttgttttttgagacagaatctcagtctcactctgtcacccaggctggagtatgttggcgtgatctctgctcactgcaacctccacctcctgggctcaagcgattctcctgcctcagcctcttgagtagctgggattgcaggtgtgcgctaccacgcatggctaatttttgtatttttaatagagatggggttttaccacgttggtcaggctggtcttgaactcctgaccttgggatccgcccacctgggcctcccaaagtgctgggattacaggcgttagccaccgcgcccAATATATTTTGATCCCTGGTTGGATATGGAGGGCTGACTGtacttaacatctctaagcttcagtttcctcctttaaaataaaggtgtggctgggtgtggtggttcaagcctgtaatcccagcacttagggaggctgaggtgggtggatcagctgaggtcaggagttcaagaccagcctgaccaatatggtgaaaccccctctctgctaaaaatacaaaaattagccaggcgtggtggcgagcgcctgtagtcccagctacttgcttgaacttgggaggcagaggttgcagtgagctgagatcgtgccactgaactcgagcatgggcaacagagcaagactgtctcaaaaaaaaaaaaaaaaaGGGGGTGAGcagacgtggtggcacgctcccacagtcccagctacttagtaggaggccaaggttggaggattgcttgatcccaggagtctgagtccagcctgggcaacatggcaatacctcatctctaaaaataaaataaaagtaaaggtattaattactactttggatggttgttgcaaagaaatatatataaaataatggagagtcttgtaactggctcccaagaggctcaacagacattACTGTTTTTGCTTCTTCATTATGAGTTACCTCTCTGGCCACCCCACTGAACTAGCTGGGCTAGCTGAGCCTGGGAGAAGAGTTGTTTAGGAAGTGAGAGGCTGCTCTCCACAGAGACTCAAGGCTCAGTTCCTCCTGGTGACTCAGATGGGCAGCCCAGTGGGCACACGTGGTCTCTCTCCACATGTGGCTGAGTTTCACTTCCAGAATAGATGGAGAGGCAAGGGCAGGGTTTAGCATGCTTGAGGAATCTCAGAGGGCCCTGGTGGTGTGGGGGACCCTCAGAACACAGGTGTCTCAAGGGCTGACCCAGCTTCTGTGTCCTTTTCTCTGGGTGAGGAGGGGACATTCATGGGCAGATGGTGACCTCTGGGGAAGGCAGCCCAGACTCCACTGGCCACCATATTTCCTTTTTCACAACTTTCTCACCCCTGTGGTTTCCCATGTCATCATGTGGCCGCTTCCCGCAAGGCCTTAGCGGGGTGCAGGTATGAACATAGTGTCAGGCAAGGAGGCATCTGGAGGGGAACCCTGGCTTTTCCTGGGGGGACTCCCTCCCTGCACCCTAGCCCTGTCCTCTCCCATGGCTACTGATGCCTTCCCCTCACCCCAGAGGTGGCCCACATCTGCACAGATCAGACCCACAAAAATCACGTCTTCCTGACTCTCATAAGCCTGCCCAGTGAGGCCCAGGCATTAGGCCATGTGCTGGGGACTCAGACCCACACATATACGCATGTCAGCATTCATGCTTACAGGTCCGCACATGCTGGGGCAAGTGTCACACACGGGGCGCTGTAGGAAGCTGACTCTCAGCCCCTGCAGATTTCTGCCTGCCTGGACAGGGAGGTGTTGAGAAGGCTCAGGCAGTCCTGGGCCAGGACCTTGGCCTGGGGCTAGGGTACTGAGTGACCCTAGAATCAAGGGTGGCGTGGGCTTAAGCAGTTGCCAGACGTTCCTTGGTACTTTGCAGGCAGACCATGTGGACCCTGGTGAGCTGGGTGGCCTTAACAGCAGGGCTGGTGGCTGGAACGCGGTGCCCAGATGGTCAGTTCTGCCCTGTGGCCTGCTGCCTGGACCCCGGAGGAGCCAGCTACAGCTGCTGCCGTCCCCTTCTGGTGAGTGCCCCTCAGCCTAGGCAAGAGCTGGCAGCCTGGGTTTTCCCAAAGGGTCATCTTGGATTGGCCAGAGGAGGACGCCAGGCACAAGTCTGTGGTTTATCATTTTCCCTGTCTTTCTAGGACAAATGGCCCACAACACTGAGCAGGCATCTGGGTGGCCCCTGCCAGGTTGATGCCCACTGCTCTGCCGGCCACTCCTGCATCTTTACCGTCTCAGGGACTTCCAGTTGCTGCCCCTTCCCAGAG'.upper()
right_context='GTAACAACTCCGTGGGTGCCATCCAGTGCCCTGATAGTCAGTTCGAATGCCCGGACTTCTCCACGTGCTGTGTTATGGTCGATGGCTCCTGGGGGTGCTGCCCCATGCCCCAGGTACAAATCtgggggagatgggggtatgtggagggaagtgggggcagagttgggggccaggggcagggggTGAAGACGGAGTCAGGACCATTTTTTCTCAGGCTTCCTGCTGTGAAGACAGGGTGCACTGCTGTCCGCACGGTGCCTTCTGCGACCTGGTTCACACCCGCTGCATCACACCCACGGGCACCCACCCCCTGGCAAAGAAGCTCCCTGCCCAGAGGACTAACAGGGCAGGTGAGGAGGTGGGAGAGCATCAGGCCAGGGGCTGGGGCGGGGCCTCATTGACTCCAAGTGTAGGAAAAAGTTTCCTCCATCCTGGCTGCCCCTCACGTTTGCTCCTCTTCCAGTGGCCTTGTCCAGCTCGGTCATGTGTCCGGACGCACGGTCCCGGTGCCCTGATGGTTCTACCTGCTGTGAGCTGCCCAGTGGGAAGTATGGCTGCTGCCCAATGCCCAACGTGAGTGAGGGGCTGGAGCCAGCTTGGCTGTGTGCCCCCAGCCACCTGGCCCTGACACGCACCTTACAGGGGCTCTGTGGCATGGGGCTGGCTGGCTGCTTGCTGGGAGCCTGGCTGATGCAGGGTTCATGCTACCCCCTAGTGGGGGATTGGGGCAGTGCCAGCCATCAGCCTGGCTGCTCCCTGTGTGCTACTGAGCCTGGAAGTGACAAAGACCCACCCCTGTCCCCACTCAGGCCACCTGCTGCTCCGATCACCTGCACTGCTGCCCCCAAGACACTGTGTGTGACCTGATCCAGAGTAAGTGCCTCTCCAAGGAGAACGCTACCACGGACCTCCTCACTAAGCTGCCTGCGCACACAGGTACCAGAGGCAGGGTGCAGATACAGGGGTGGGGCCCCCTTTCCTCCCTTTTAGGCCTGGCCTTAGGATCACTGCAAGGTGGTGTAAGCGGTACCCTCCATCTTCAACACCTGGTTCCAGCTGTGGAGCCGGCAAAGGGTTGATACCCCTGAGGGTCCCCAGTGCCACTTCTGACCTGTCCTCTCTGCTTCCCTCACAGTGGGGGATGTGAAATGTGACATGGAGGTGAGCTGCCCAGATGGCTATACCTGCTGCCGTCTACAGTCGGGGGCCTGGGGCTGCTGCCCTTTTACCCAGGTACCCAGGGGTGGCGGGTGGGTGGGCTGAGCACAGTGTGGCAGGCAGCCGGGCCCCAGTGCCCACCTGCCCTTCTTCATCTGCCCTAGGCTGTGTGCTGTGAGGACCACATACACTGCTGTCCCGCGGGGTTTACGTGTGACACGCAGAAGGGTACCTGTGAACAGGGGCCCCACCAGGTGCCCTGGATGGAGAAGGCCCCAGCTCACCTCAGCCTGCCAGACCCACAAGCCTTGAAGAGAGATGTCCCCTGTGATAATGTCAGCAGCTGTCCCTCCTCCGATACCTGCTGCCAACTCACGTCTGGGGAGTGGGGCTGCTGTCCAATCCCAGAGGTATATGGGAGGGGACAGCATCTTGGCCTGGGCAGGTGGGTGGCCAAGCTCCTATTGCTTTCTGCCCTCCGCATAGCCCATAGGTGATACCCAGCTCTGACAGATTCGTCCCCAGCTGGAGGTGCTGTAAGCAGGAGAGGCGGGCTGGAGTAGGTAGGGGCTCGGCACTGCGCCCCACATAGTGGCTACCTACAACGCCCTTTCCTGCCCACCCCCCAGGCTGTCTGCTGCTCGGACCACCAGCACTGCTGCCCCCAGGGCTACACGTGTGTAGCTGAGGGGCAGTGTCAGCGAGGAAGCGAGATCGTGGCTGGACTGGAGAAGATGCCTGCCCGCCGGGCTTCCTTATCCCACCCCAGAGACATCGGCTGTGACCAGCACACCAGCTGCCCGGTGGGGCAGACCTGCTGCCCGAGCCTGGGTGGGAGCTGGGCCTGCTGCCAGTTGCCCCATGTGAGTGCCTCCCTGCCTGCCCCTGGATAGGGGAGCTAAGCCCAGTGAGGGGACAGGAACATAATGCCATTCTGTGCTCCCTTCCCCGCCAGGCTGTGTGCTGCGAGGATCGCCAGCACTGCTGCCCGGCTGGCTACACCTGCAACGTGAAGGCTCGATCCTGCGAGAAGGAAGTGGTCTCTGCCCAGCCTGCCACCTTCCTGGCCCGTAGCCCTCACGTGGGTGTGAAGGACGTGGAGTGTGGGGAAGGACACTTCTGCCATGATAACCAGACCTGCTGCCGAGACAACCGACAGGGCTGGGCCTGCTGTCCCTACCGCCAGGTCAGTGCCAACCCCCATCCTGGGGCTGGGTATGGCCAGGGACCAGGTCCCACCTCGTCCAACCCTCTCGCCCCCCTCTGACCATCCAGGGCGTCTGTTGTGCTGATCGGCGCCACTGCTGTCCTGCTGGCTTCCGCTGCGCAGCCAGGGGTACCAAGTGTTTGCGCAGGGAGGCCCCGCGCTGGGACGCCCCTTTGAGGGACCCAGCCTTGAGACAGCTGCTGTGAGGGACAGTACTGAAGACTCTGCAGCCCTCGGGACCCCACTCGGAGGGTGCCCTCTGCTCAGGCCTCCCTAGCACCTCCCCCTAACCAAATTCTCCCTGGACCCCATTCTGAGCTCCCCATCACCATGGGAGGTGGGGCCTCAATCTAAGGCCTTCCCTGTCAGAAGGGGGTTGTGGCAAAAGCCACATTACAAGCTGCCATCCCCTCCCCGTTTCAGTGGACCCTGTGGCCAGGTGCTTTTCCCTATCCACAGGGGTGTTTGTGTGTGTGCGCGTGTGCGTTTCAATAAAGTTTGTACACTTTCTTAACAGTGTCTGATTTGCCGCCCTGCCTGCCCTCCCCAGGGCCCCAGAACAGGGGTTCACGTCCACTGCCAACACTCCCCTCCCCTACCCCACAGAAAGACATACACAGCCTTAACCTCACCAGTTTTATATTTGCTGCTGCCCACATCGGCTGTATCACATTACCCCCTAGTCCCCAGAGCTGTCCCAGCCACCAGCCCTGTGACATCGTAGCCCAGAGATGGGCTAGTCAGCAAGCAGCACCCCCTCCCCTCCCAGGGGTCCACAAAGAACGCCCCCTCCCTTCCCAGCCCTCACACTAGCAGCTGAGGCTGGGTCACCCCTCCTGCTTTCCCACAATAGAGCTTTCTATGTACAGCCACGTCTACACAGGCACTGCTTCcccccagccctcctccccggcacctccccgtgggggtctggaccccccctcctccccGGCTTGGAGGCAGACACAGGGTCCCTTGCAAGACACGACCCAGCACCAACCACGGAACAGCTCCAAGGCCCCTGGGCCCCTCTCCGGCCTGGGGCTGGGAGCTacgcgcgagggcccccgcgggcccccggggcgcgcaccctgggtgcgggcccgcgcgggaggggCGGTGCCAGGCCCTGCGCGGGCGCTACTTGACGTTGAACACCATCAGCGGCCGCTCCTCCCGCCGCTGCCACGGGCTCTTCTTGTCTCCCGCCTCGTCGCCCACCTCCGCCCCCAGCTCGTCCTCCGGGCTGGTGAGCGAGTCGCGCCGCAGCTCGCTGGCGCTGCTGCGGGAGCTGTCCCCGCGGCTGCGGCCCCGCCCCCGGGGTACCGTGGCCGCCCGGCCCTCGGGCGCCGCCACCTCGTCCAGCAGCGGCGTCAGCGAGTTGTCCTCCGGAGAGCAGAGGCCCGTGCGGCTCTCGCTGCTGCTGGGCTCCGTGTCCTCGCTGAgcgccaggcgcgggggcgcgggcggcggcggcggcgcggcaggcggggcgcgctcccgctcctccctcgcgggccggcggcgcgcggGCCGCGCCTCGTGGACATCTACGCCCGAGTCCAGGCTGGCGTCGGTGCTGCGTGCCCCGCCGCCCGCCTGCAGGTCGATGTAAGAGTGGCGCACTTGCGAGTTGGAGCGCCCGTCGAGGGACACGAACCAGGCGCGCGGGTGCGGCTTCACGCCCAGTTCCAGCAGCTTCTTCTCGGTCAGGGCCTGCAGCTCCCCGTTGAGCTGCGCCATGGTGGACTCGTTGAATAGCACAGGGATGGTGACTGAGCCACTGACGGGcgccgcgcgcccggccccccagccctcgccgccgcccccgccgccctcgccccccgggcccgagtggcccgGCATCTGCGGGCGCTGGGGGTCGGGCTGGGGAAAAGCGCGCGCCGGGCCGGGTGCCGTGCCCTCCGGCGGGGCCGGCTCGTCGCCCACGCCGGCGGCGCCCGCCTCGCCGCCGAGGCGCACGTAGTGCGCGGGGATCACCAGGGTGGGCATGACGTTGCGGTAGACGTTGTCCTTGAGGTGGTCGATGGAGCCGCAGAAGATGAGCTGCCCCGCCTGGCCCAGCGACGGCGGCCGCGCCAGCTGGTCCACCGACTGCGACAGCAGGAAGTCGGGGGTCTTGCCCTCGGCCGCCCCCTTGTGGCCCAGGTAGTGGTCGAAGGGCGGCGGCGGCGAGGGCGGCTCGTGCAGGAAGGCCGCAGCCCCCGAGGGCCCCCGCCGGTGCTCCTCTAGGCCGGGCTCCAGCCCGCCGGGGCCCTCGGCCGAGCGAGCGCCCTTGAGCCCGGCGCTCTCGCCCCCGCGGGCACCCGAAGGCTCGGCGGCCGGGCGGCTGGCAGAGCGCGGCTTGGTGCGGAAGAAGTCATCCCGGGAGGAGGCCAAGTCCCGGGAGCTGGAGAAGGCCGAGTGGAGGGGGCCTGGAGGCGGAGCCTCGGGGTCCCCCGACGGGGCGGGTTCCAGGGGTCCCCCACAGATGAGGTGGAGCTGGGACATCGAGGTGGCCTGGTCTCGTTTGTTACCGTCAGAGGGCCCCGAGAGCTGCAGCTTGCGGTGCTGTTGCCTCGGCTTCAGGCAGCGCCTCCTGGAAGGGAGGGAGCAGAAGGGGCCGCTCAGGAAA'.upper()
intron_exon_intron = 'GTGAGCGTGCCATCAGCCCAGTGGAGGGGCTTAGGTCTGCATTTATGCTTTTCCTGCACTCTACCACCTGCAGATAAAAGGGCCCTGCCAATGCAGGTTTCTCTGTGTTCCACAGGCCGTGGCATGCGGGGATGGCCATCACTGCTGCCCACGGGGCTTCCACTGCAGTGCAGACGGGCGATCCTGCTTCCAAAGATCAGGTGCAGCTGGGGTGTGGGTGCAGGGCAGGCAGACGGGCAGCATGTGGAGTCTGGAACCCAGGAGCCCAGCTGGCGGGGGCAGCCCTGATTCCTGCCCTTGTGCCCTCATTCATGTGGCATCTGTACTAAGCAACAGCCCTGCTGTGGACAGAGGGGCAGCACTGGGGATAGGAGGGTGCGGGAGAAAGTGCAAGACTCCAGGTCCAGGCGTTGTGGGGGTGGGGAGAGGTCGAGCTGGGCCGGTCTAATACCAACCCATGGTCAGTGGGTGCCCCTTCCCCATGCCATCTTGCTGAGGGAGGGACTGGATTGTGAGGAGGGTGAGTTAGGCCTGCCTAGGAGATCACTGAGCCTTAGTGTCACCCTCAAACCCCAGTAGCTGGGCTTGCAGGCCCTGGTGCCACCAGCTCCTTGTGTGATGGGGGAGTCACCTTCCCTGAGTGGGCTGGTAGTATCCTGGGTCATCTTGTCCACAG'.upper()

full_seq = left_context + intron_exon_intron + right_context

exon_seq = 'GCCGTGGCATGCGGGGATGGCCATCACTGCTGCCCACGGGGCTTCCACTGCAGTGCAGACGGGCGATCCTGCTTCCAAAGATCAG'
ASO_seq = 'GGAACCCAGGAGCCCAG'


acceptor_pos=5115 # full_seq에서 acceptor 위치
donor_pos=5199 # (10769 + len(exon_seq)-1), full_seq에서 donor 위치


In [5]:
full_seq[acceptor_pos-2:acceptor_pos]

'AG'

In [6]:
full_seq[donor_pos+1:donor_pos+3]

'GT'

In [7]:
len(full_seq)

10676

In [9]:
def get_prob_with_seq(seq):
    input_x = one_hot_encode(seq)[None, :]
    output_y = np.mean([spliceai_models[m].predict(input_x) for m in range(5)], axis=0)
    return output_y

In [10]:

all_result=[]

input_seq=full_seq
len_of_aso=17


mask_seq='N'*17

best_total=0.0

for now_mut_pos in range(acceptor_pos-5030,donor_pos+5020):
    
    original_seq = input_seq
    masked_seq = input_seq[:now_mut_pos] + mask_seq + input_seq[now_mut_pos+len_of_aso:]


    original_donor_seq = original_seq[donor_pos-5000:donor_pos+5000+1]
    original_acceptor_seq = original_seq[acceptor_pos-5000:acceptor_pos+5000+1]


    masked_donor_seq = masked_seq[donor_pos-5000:donor_pos+5000+1]
    masked_acceptor_seq = masked_seq[acceptor_pos-5000:acceptor_pos+5000+1]


    original_donor_prob = get_prob_with_seq(original_donor_seq)[0][0][2]
    original_acceptor_prob = get_prob_with_seq(original_acceptor_seq)[0][0][1]

    masked_donor_prob = get_prob_with_seq(masked_donor_seq)[0][0][2]
    masked_acceptor_prob = get_prob_with_seq(masked_acceptor_seq)[0][0][1]

    
    
    
    result_total = (masked_donor_prob+masked_acceptor_prob)/2.0
    delta_acceptor = masked_acceptor_prob - original_acceptor_prob
    delta_donor = masked_donor_prob - original_donor_prob
    delta_total = delta_acceptor + delta_donor
    delta_pos = now_mut_pos
    erased_seq = input_seq[now_mut_pos:now_mut_pos+17]

    all_result.append( [delta_total,delta_acceptor,delta_donor,delta_pos,erased_seq] )
        
        
    f = open("ASO_grn_result_renew.txt", 'a')
    
    def inline_str(str_list):
        num = len(str_list)
        ret=""
        for i in range(num):
            ret+=str(str_list[i])
            if i<num-1:
                ret+='\t'
        ret+='\n'
        return ret
        
    
    f.write(inline_str([result_total,delta_acceptor,delta_donor,delta_total,delta_pos,erased_seq,original_acceptor_prob,original_donor_prob,masked_acceptor_prob,masked_donor_prob]))
    f.close()
    
    if now_mut_pos%20==0:
        print('done '+str(now_mut_pos),end="  /  ")
        

done 100  /  done 120  /  done 140  /  done 160  /  done 180  /  done 200  /  done 220  /  done 240  /  done 260  /  done 280  /  done 300  /  done 320  /  done 340  /  done 360  /  done 380  /  done 400  /  done 420  /  done 440  /  done 460  /  done 480  /  done 500  /  done 520  /  done 540  /  done 560  /  done 580  /  done 600  /  done 620  /  done 640  /  done 660  /  done 680  /  done 700  /  done 720  /  done 740  /  done 760  /  done 780  /  done 800  /  done 820  /  done 840  /  done 860  /  done 880  /  done 900  /  done 920  /  done 940  /  done 960  /  done 980  /  done 1000  /  done 1020  /  done 1040  /  done 1060  /  done 1080  /  done 1100  /  done 1120  /  done 1140  /  done 1160  /  done 1180  /  done 1200  /  done 1220  /  done 1240  /  done 1260  /  done 1280  /  done 1300  /  done 1320  /  done 1340  /  done 1360  /  done 1380  /  done 1400  /  done 1420  /  done 1440  /  done 1460  /  done 1480  /  done 1500  /  done 1520  /  done 1540  /  done 1560  /  done 1580

In [28]:
len(input_seq)-len_of_aso+1

6251

In [45]:
f = open("ASO_result.txt", 'r')

In [46]:
res=[]


for line in f:
    p=line.split('\t')
    res.append([float(p[0]),p[4],int(p[3])])


In [47]:
res.sort()
res[-10:]

[[0.28564858, 'CCAGCATTATGAAAGTG', 5832],
 [0.2863837, 'TTAGACAAAATCAAAAA', 5773],
 [0.28948206, 'GTGCTCACATTCCTTAA', 5798],
 [0.2931587, 'TGCCAGCATTATGAAAG', 5830],
 [0.3000751, 'TTTTAGACAAAATCAAA', 5771],
 [0.30198628, 'GTTTTAGACAAAATCAA', 5770],
 [0.3060084, 'CATTCCTTAAATTAAGG', 5805],
 [0.30910528, 'CTGCCAGCATTATGAAA', 5829],
 [0.31307673, 'ACATTCCTTAAATTAAG', 5804],
 [0.3230291, 'ATTCCTTAAATTAAGGA', 5806]]

In [149]:
def predict_mutation(input_seq, donor_pos, acceptor_pos, len_of_aso ,use_n_as_aso=True):
    
#     total_delta=0.8
#     acceptor_delta=0.5
#     donor_delta=0.3
#     delta_pos=536
#     erased_seq='ATTTGGCCAAGCGATGC'
    
    
#     all_result.append([[total_delta,acceptor_delta,donor_delta,delta_pos,erased_seq]])
    
    
    all_result=[]
    
    
    
    if use_n_as_aso:
        mask_seq='N'*len_of_aso
    else:
        mask_seq=make_random_seq(len_of_aso)
    
    best_total=0.0
    
    for now_mut_pos in range(len(input_seq)-len_of_aso+1):
        original_seq = input_seq
        masked_seq = input_seq[:now_mut_pos] + mask_seq + input_seq[now_mut_pos+len_of_aso:]
        
        original_seq = left_context + original_seq + right_context
        masked_seq = left_context + masked_seq + right_context
        
        
        original_donor_seq = original_seq[donor_pos-5000:donor_pos+5000+1]
        original_acceptor_seq = original_seq[acceptor_pos-5000:acceptor_pos+5000+1]
        
        
        masked_donor_seq = masked_seq[donor_pos-5000:donor_pos+5000+1]
        masked_acceptor_seq = masked_seq[acceptor_pos-5000:acceptor_pos+5000+1]
        

        original_donor_prob = get_prob_with_seq(original_donor_seq)[0][0][2]
        original_acceptor_prob = get_prob_with_seq(original_acceptor_seq)[0][0][1]
        
        masked_donor_prob = get_prob_with_seq(masked_donor_seq)[0][0][2]
        masked_acceptor_prob = get_prob_with_seq(masked_acceptor_seq)[0][0][1]
        
        
        delta_acceptor = masked_acceptor_prob - original_acceptor_prob
        delta_donor = masked_donor_prob - original_donor_prob
        delta_total = delta_acceptor + delta_donor
        delta_pos = now_mut_pos
        erased_seq = input_seq[now_mut_pos:now_mut_pos+17]
        
        all_result.append( [delta_total,delta_acceptor,delta_donor,delta_pos,erased_seq] )
      
        best_total = max(best_total,delta_total)
    
        if now_mut_pos%20==0:
            print('done '+str(now_mut_pos),end="  /  ")

In [ ]:
predict_mutation(intron_exon_intron,donor_pos,acceptor_pos,17)

done 0  /  done 10  /  done 20  /  done 30  /  done 40  /  done 50  /  done 60  /  done 70  /  done 80  /  done 90  /  done 100  /  done 110  /  done 120  /  done 130  /  done 140  /  done 150  /  done 160  /  done 170  /  done 180  /  done 190  /  done 200  /  done 210  /  done 220  /  done 230  /  done 240  /  done 250  /  done 260  /  done 270  /  done 280  /  done 290  /  done 300  /  done 310  /  done 320  /  done 330  /  done 340  /  done 350  /  done 360  /  done 370  /  done 380  /  done 390  /  done 400  /  done 410  /  done 420  /  done 430  /  done 440  /  done 450  /  done 460  /  done 470  /  done 480  /  done 490  /  done 500  /  done 510  /  done 520  /  done 530  /  done 540  /  done 550  /  done 560  /  done 570  /  done 580  /  done 590  /  done 600  /  done 610  /  done 620  /  done 630  /  done 640  /  done 650  /  done 660  /  done 670  /  done 680  /  done 690  /  done 700  /  done 710  /  done 720  /  done 730  /  done 740  /  done 750  /  done 760  /  done 770  

In [ ]:
5초에 10개

In [ ]:
predict_mutation(intron_exon_intron,donor_pos,acceptor_pos,17)